In [ ]:
import os
import json
import random
from tqdm import tqdm
import pickle

from collections import defaultdict

## merge NY & LA dataset

In [ ]:
# merge
with open('raw/tweets-la.json') as f:
    readin1 = f.readlines()
    
with open('raw/tweets-ny.json') as f:
    readin2 = f.readlines()
    
readin = readin1 + readin2
print(len(readin1))
print(len(readin2))
print(len(readin))

print(readin[0])
random.shuffle(readin)
print(readin[1])

print(len(readin))

with open('tweets.json','w') as fout:
    for line in readin:
        fout.write(line)

## data statistics

In [ ]:
data = []

with open('tweets.json') as f:
    readin = f.readlines()
    for line in tqdm(readin):
        data.append(json.loads(line))

print(len(data))

In [ ]:
for i in range(100):
    print(data[i])

In [ ]:
tweet_text_dict = {} # key:tweet_id, value: tweet_text
tweet_author_dict = {} # key:tweet_id, value: author_id
tweet_poi_dict = {} # key:tweet_id, value: poi_id
tweet_tags = {} # key:tweet_id, value: tag list
tweet_mentions = {} # key:tweet_id, value: mentions list

poi_text_dict = {} # key:poi_id, value: text
poi_label_dict = {} # key:poi_id, value: poi_label
poi_tweet_dict = defaultdict(list) # key:poi_id, value: tweet_id list

author_tweet_dict = defaultdict(list) #key: author_id, value: tweet_id list
tag_tweet_dict = defaultdict(list) #key: tag_name, value: tweet_id list
mention_tweet_dict = defaultdict(list) #key: mention_name, value: tweet_id list

label_poi_dict = defaultdict(list) #key: label_name, value: poi_id list

for d in tqdm(data):
    tweet_text_dict[d['tweet_id']] = d['tweet_text']
    tweet_author_dict[d['tweet_id']] = d['author_id']
    tweet_poi_dict[d['tweet_id']] = d['poi_id']
    tweet_tags[d['tweet_id']] = d['tags']
    tweet_mentions[d['tweet_id']] = d['mentions']
    
    for t in d['tags']:
        tag_tweet_dict[t].append(d['tweet_id'])
    for m in d['mentions']:
        mention_tweet_dict[m].append(d['tweet_id'])
    label_poi_dict[d['label']].append(d['poi_id'])
    
    poi_text_dict[d['poi_id']] = d['poi_text']
    poi_label_dict[d['poi_id']] = d['label']
    poi_tweet_dict[d['poi_id']].append(d['tweet_id'])
    
    author_tweet_dict[d['author_id']].append(d['tweet_id'])

print(f'Number of tweet:{len(tweet_text_dict)}, Number of POI:{len(poi_text_dict)}, Number of labels:{len(label_poi_dict)}')
print(f'Number of tags:{len(tag_tweet_dict)}, Number of mentions:{len(mention_tweet_dict)}, Number of authors:{len(author_tweet_dict)}')

In [ ]:
# statistics on tags
summ = 0
thresh = 3
cnt = 0

for t in tag_tweet_dict:
    summ += len(tag_tweet_dict[t])
    if len(tag_tweet_dict[t]) >= thresh:
        cnt += 1
    #print(len(tag_tweet_dict[t]))
print(f'Average:{summ/len(tag_tweet_dict)}, Cnt:{cnt}')

In [ ]:
# statistics on mentions
summ = 0
thresh = 3
cnt = 0

for t in mention_tweet_dict:
    summ += len(mention_tweet_dict[t])
    if len(mention_tweet_dict[t]) >= thresh:
        cnt += 1
    #print(len(mention_tweet_dict[t]))
print(f'Average:{summ/len(mention_tweet_dict)}, Cnt:{cnt}')

In [ ]:
# statistics on authors
summ = 0
thresh = 3
cnt = 0

for t in author_tweet_dict:
    summ += len(author_tweet_dict[t])
    if len(author_tweet_dict[t]) >= thresh:
        cnt += 1
    #print(len(author_tweet_dict[t]))
print(f'Average:{summ/len(author_tweet_dict)}, Cnt:{cnt}')

In [ ]:
# statistics on O
summ = 0
thresh = 2
cnt = 0

for t in poi_tweet_dict:
    summ += len(poi_tweet_dict[t])
    if len(poi_tweet_dict[t]) >= thresh:
        cnt += 1
    #print(len(poi_tweet_dict[t]))
print(f'Average:{summ/len(poi_tweet_dict)}, Cnt:{cnt}')

In [ ]:
# twitter text length
summ = 0
for t in tweet_text_dict:
    summ += len(tweet_text_dict[t].split())
print(summ/len(tweet_text_dict))

In [ ]:
# poi text length
summ = 0
for t in poi_text_dict:
    summ += len(poi_text_dict[t].split())
print(summ/len(poi_text_dict))    

## Seperate train/val/test

In [ ]:
with open('tweets.json') as f:
    readin = f.readlines()
print(len(readin))

In [ ]:
# seperate data
train_ratio = 0.7
val_ratio = 0.1

train_set = readin[:int(len(readin)* train_ratio)]
val_set = readin[int(len(readin)* train_ratio):int(len(readin)* (train_ratio+val_ratio))]
test_set = readin[int(len(readin)* (train_ratio+val_ratio)):]

In [ ]:
# save file
with open('raw/train.tsv','w') as fout:
    for line in train_set:
        fout.write(line)

with open('raw/val.tsv','w') as fout:
    for line in val_set:
        fout.write(line)
        
with open('raw/test.tsv','w') as fout:
    for line in test_set:
        fout.write(line)